# Prototyping variability_selection

Goal: figure out precisely which criteria we will use to select "valid" stars (Q2, Q1, etc) and 
One change I am considering making: expanding the "Q1" criterion to include stars who may not have any 100% bands, but who are like 90% good in each band. (I'd like to compute variability stats on ONLY their `good` data in these cases, which might require adding some columns to spreadsheet_maker.)

One consideration: backwards compatibility, at least with the 2015 orion paper. At the very very least, let's confirm that we can reproduce those results to some degree.

I'm looking at ["official_star_counter" from wuvsars-orion](https://github.com/tomr-stargazer/wuvars-orion/blob/master/official_star_counter.py).

(We'll also, someday, be interested in splitting off WSERV5-SE and treating it as its own thing, too.)

### First: Can we, like, re-run wuvsars-orion's official star counter?

official star counter lives here:
/Users/tsrice/Documents/Code/wuvars-orion/official_star_counter.py


In [2]:
%run /Users/tsrice/Documents/Code/wuvars-orion/official_star_counter.py

Auto-detected table type: fits
Auto-detected table type: fits
Auto-detected table type: fits
Auto-detected table type: fits
Number of detected sources in the dataset:
40630
Number of stars that meet absolute minimum considerations for valid data:
(i.e., have at least 50 recorded observations in at least one band)
14728
Maximum possible number of variables: 3141
Number of stars automatically classed as variables: 868
Number of stars that have the data quality for auto-classification: 3592
Auto-detected table type: fits

Number of probably-variable stars requiring subjective verification due to imperfect data quality: 2273
Number of new subjectives: 94

Number of STRICT autovariables: 553
Number of STRICT autocandidates: 2348

 Q: Statistically, what fraction of our stars are variables?
 A: 23.55%, drawn from the tightest-controlled sample;
    24.16%, drawn from a looser sample.

Number of possible variables with detected periods: 585
Number of autovariables that are periodic: 354
Numbe

### Stats from "old" official star counter:

- Q0 stars (at least 50 observations in at least one band): 
 - 14728
- Total detected sources:
 - 40630
- Q2 stars:
 - 2348
- Q1 + Q2 stars:
 - 3592

# Question 1: 

Given that we've shifted away from old "summary spreadsheet" code from ~2012 (which used ATpy internally) to new code which uses Pandas internally (for a huge boost in performance, maintainability/clarity, and compatibility with Python 3), can we reproduce the numbers from Table 1 of Rice et al 2015? In other words, **can we verify that the new code produces the same output as the old code**, given the same photometric data and the same definitions for "quality bins"?

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.table import Table


In [8]:
# Let's re-implement the above for WSERV5, in my new reduction.

spreadsheet_root = "/Users/tsrice/Documents/Variability_Project_2020/wuvars/Data/analysis_artifacts"
wserv_ids = [5]

for wserv in wserv_ids[::-1]:
    
    print(f"\n   WSERV{wserv}: \n")

    spreadsheet_path = os.path.join(
        spreadsheet_root,
        f"wserv{str(wserv)}",
        f"WSERV{str(wserv)}_graded_clipped0.95_scrubbed0.1_dusted0.5_summary_spreadsheet.h5",
    )
    if wserv==5:
        spreadsheet_path = os.path.join(
            spreadsheet_root, 
            "wserv5_v2012",
            f"WSERV{str(wserv)}_fdece_graded_clipped0.8_scrubbed0.1_dusted0.5_summary_spreadsheet.h5")
        print(f"WSERV5: {spreadsheet_path}")
    
    ds = pd.read_hdf(spreadsheet_path, key='table')

    q0 = (
        (ds["count"]["N_J"] >= 50)
        | (ds["count"]["N_H"] >= 50)
        | (ds["count"]["N_K"] >= 50)
    )
    
    print("Total detected sources:", len(ds))
    print("Total sources with at least 50 obs in one band:", len(ds[q0]))



   WSERV5: 

WSERV5: /Users/tsrice/Documents/Variability_Project_2020/wuvars/Data/analysis_artifacts/wserv5_v2012/WSERV5_fdece_graded_clipped0.8_scrubbed0.1_dusted0.5_summary_spreadsheet.h5
Total detected sources: 40630
Total sources with at least 50 obs in one band: 15101


# Answer to Question 1:

Okay, the good: we are picking up **exactly** the same number of detected sources for WSERV5 as before. (40630)

(Context: this is the version of the spreadsheet which uses the old, '80% graded' data, as an exact copy from 2012.)

The mostly-good: we are picking up very nearly the same number of Q0 sources (15,101 versus the old 14,728). I'm not sure where these 373 newcomers came from, but I'm not particularly invested in finding out.

One might consider it a *problem* that these are not the exact same value. Can we modify the query in some way to get it to be identical? We seem to have *too many* sources in our new criterion.

I think the difference might be that I've computed my N_J in a way that is not treating nulls properly. But - that doesn't really make sense. 

The appropriate way to investigate this further would be to actually *look* at the data for the 373 stars which are excluded by the old code but included in the new code. That would be really valuable, actually. How can we get our hands on those data?

In [4]:
15101 - 14728

373

In [27]:
print(len(minimum.SOURCEID))
print(len(ds[q0]))

# This is how you figure out the members of one set that are not members of the other set
np.sum(~np.in1d(ds[q0].index, minimum.SOURCEID))

new_sourceids = ds[q0].index[~np.in1d(ds[q0].index, minimum.SOURCEID)]

print(new_sourceids)


14728
15101
Int64Index([44199508443330, 44199508444143, 44199508444192, 44199508444533,
            44199508444815, 44199508445147, 44199508446658, 44199508447195,
            44199508447196, 44199508447525,
            ...
            44199508575548, 44199508576260, 44199508576676, 44199508577218,
            44199508577222, 44199508577223, 44199508577225, 44199508577227,
            44199508577228, 44199508577383],
           dtype='int64', name='SOURCEID', length=373)


## We've identified the SOURCEIDs of 373 stars that are picked up as "Q=0" in the new code but not the old code.

What's going on here? Is there a bug or logical error in either the new or old spreadsheet code? How could something as simple as "count how many nights of data there are for a given star" have any ambiguity?

The plan: let's look at those stars individually (at least a couple of them).

In [29]:
# The photometry data lives here...

filename = "/Users/tsrice/Documents/Variability_Project_2020/wuvars/Data/copied_from_old_projects/WSERV5_fdece_graded_clipped0.8_scrubbed0.1_dusted0.5.fits"

dat = Table.read(filename)
df = dat.to_pandas()


In [38]:
pd.set_option('display.max_rows', 30)
first_new_source_photometry = df[df['SOURCEID'] == new_sourceids[0]]

first_new_source_photometry.head(30)

,SOURCEID,MEANMJDOBS,RA,DEC,JMHPNT,JMHPNTERR,HMKPNT,HMKPNTERR,JAPERMAG3,JAPERMAG3ERR,...,KAPERMAG3,KAPERMAG3ERR,JPPERRBITS,HPPERRBITS,KPPERRBITS,MERGEDCLASS,PSTAR,JGRADE,HGRADE,KGRADE
832,44199508443330,54034.538810,1.463295,-0.091874,-9.999995e+08,1.040192e+09,-9.999995e+08,1.040192e+09,-9.999995e+08,1.040192e+09,...,-9.999995e+08,1.040192e+09,0,64,0,1,0.050000,0.000000,0.642857,0.000000
833,44199508443330,54035.511963,1.463295,-0.091874,-9.999995e+08,1.040192e+09,-9.999995e+08,1.040192e+09,-9.999995e+08,1.040192e+09,...,1.519227e+01,2.766662e-02,0,0,4194320,1,0.050000,1.000000,0.996226,0.970370
834,44199508443330,54039.487877,1.463296,-0.091874,-9.999995e+08,1.040192e+09,1.189989e+00,4.171116e-02,-9.999995e+08,1.040192e+09,...,1.534266e+01,2.883212e-02,0,4194320,4194320,1,0.003067,0.843023,0.988722,0.981481
835,44199508443330,54040.528303,1.463295,-0.091874,1.873665e+00,1.230536e-01,1.296405e+00,4.500960e-02,1.842997e+01,1.182191e-01,...,1.525990e+01,2.931586e-02,4194304,4194320,4194320,1,0.002915,0.988506,0.973585,0.974265
836,44199508443330,54050.517097,1.463296,-0.091873,-9.999995e+08,1.040192e+09,1.479374e+00,4.941564e-02,-9.999995e+08,1.040192e+09,...,1.525394e+01,2.857197e-02,0,4194320,4194320,1,0.003067,0.994350,0.992453,0.970588
837,44199508443330,54051.567163,1.463296,-0.091873,-9.999995e+08,1.040192e+09,-9.999995e+08,1.040192e+09,-9.999995e+08,1.040192e+09,...,1.516781e+01,2.897146e-02,0,0,4194320,1,0.050000,0.977528,0.905405,0.935294
838,44199508443330,54052.557993,1.463295,-0.091873,-9.999995e+08,1.040192e+09,-9.999995e+08,1.040192e+09,-9.999995e+08,1.040192e+09,...,1.529272e+01,2.804404e-02,0,0,4194320,1,0.050000,1.000000,0.988679,0.949091
839,44199508443330,54053.518190,1.463295,-0.091874,-9.999995e+08,1.040192e+09,-9.999995e+08,1.040192e+09,-9.999995e+08,1.040192e+09,...,1.534746e+01,2.926424e-02,0,0,4194320,1,0.050000,0.000000,0.000000,0.963636
840,44199508443330,54053.554475,1.463296,-0.091874,-9.999995e+08,1.040192e+09,-9.999995e+08,1.040192e+09,1.839043e+01,1.068936e-01,...,-9.999995e+08,1.040192e+09,4194304,0,0,1,0.050000,1.000000,0.962121,0.000000
841,44199508443330,54054.564130,1.463295,-0.091874,-9.999995e+08,1.040192e+09,-9.999995e+08,1.040192e+09,-9.999995e+08,1.040192e+09,...,1.535931e+01,2.897304e-02,0,0,4194320,1,0.050000,0.971264,0.992481,0.974265


# Initial thoughts

Okay, upon reviewing the above object... I'm trying to figure out which of its bands could have been plausibly included in our "at least 50 observations in at least one band" criteria in the first place. Let's do some calculations.

In [40]:
np.sum([first_new_source_photometry['KAPERMAG3'] > 0]) #okay that checks out, actually!!! why didn't it get picked up before??

65

In [41]:
print(np.sum([first_new_source_photometry['HAPERMAG3'] > 0]) )
print(np.sum([first_new_source_photometry['JAPERMAG3'] > 0]) )

29
5


# Further thoughts

With this one source specifically above, we're seeing that it has enough K band observations (with real photometry, not negative-a-billion null values) that in my opinion it should be (and should have been) counted as a Q=0 star. Nonetheless, its K band photometry is garbage - its KPPERRBITS column (everywhere that the KAPERMAG3 *isn't* negative a billion) appears to be 4194320, a value that indicates "Source lies within a dither offset of the frame boundary" according to the WFCAM Science Archive [Quality Error Bit Flags](http://wsa.roe.ac.uk/ppErrBits.html#Source_image_close_to_frame_boun) page.

Now, given how garbage its K data are, I think I agree that it shouldn't count as a Q0 star (or, more specifically, it would be wiser to exclude it). But - why did it get excluded earlier, yet included now? As far as I can tell, the old criterion looks like this:

```python
# Stars with valid data (that could be considered candidates for inclusion)
# Criteria:
#  At least 50 observations (as measured by Stetson_N or just per band)
#  

minimum = spread.where((spread.N_j >= 50) |
                       (spread.N_k >= 50) |
                       (spread.N_h >= 50) )
```

whereas the new criterion (literally stated above) looks like this:

```python
    q0 = (
        (ds["count"]["N_J"] >= 50)
        | (ds["count"]["N_H"] >= 50)
        | (ds["count"]["N_K"] >= 50)
    )
```

How are these *not* the same exact criteria? What's going on?

# What's going on

Here I'm going to take a look at how exactly the old and new codes compute `N_j` or `N_J` given a table of photometry data.

This is detective work.

Old Code:

`wuvars-orion/variability_script_orion.py:`
```python
        # any cuts on the data.
        sp_i = sp.spreadsheet_write(data_i, lookup_i, -1, 
                                    path2+'sp%d.fits'%i, flags=256,
                                    per=True, graded=True, rob=True,
                                    colorslope=True)
```

`wuvars-proto/tr/spread3.py:`
```python
def spreadsheet_write (table, lookup, season, outfile, flags=0,
                       nowrite=False, Test=False,
                       rob=False, per=False, graded=False, colorslope=False):
    ...
    for sid, i in zip(sidarr, list(range(len(sidarr))) ):

        # v for values
        v = statcruncher (table, sid, season, rob, per, graded=graded,
                          flags=flags, colorslope=colorslope)
        if v == None:
    ...
    
def statcruncher (table, sid, season=0, rob=True, per=True, 
                  graded=False, colorslope=False, flags=0) :
    j_table = band_cut(s_table, 'j', max_flag=flags)
    h_table = band_cut(s_table, 'h', max_flag=flags)
    k_table = band_cut(s_table, 'k', max_flag=flags)     
```

`wuvars-proto/tr/helpers3.py`
```python
def band_cut (table, band, min_flag=0, max_flag=2147483648,
              null=np.double(-9.99999488e+08)): 
    ...
    if len(band) == 1:
        cut_table = table.where( (table.data[band_name] != null) &
                                 (table.data[banderr_name] != null) &
                                 (table.data[pperrbits_name] >= min_flag) &
                                 (table.data[pperrbits_name] <= max_flag) )
    else:
        cut_table = table.where( (table.data[band_name] != null) &
                                 (table.data[banderr_name] != null) )
        

    return cut_table    
```

In [49]:
# Also: let's look at the actual stats recorded in the summary spreadsheet for this same object!

spread.N_k[spread.SOURCEID == new_sourceids[0]]

array([3])

# What to do from here

I think I need to modify my program, `wuvars/analysis/create_summary_spreadsheets.py`, to also treat the data with a `max_flags` keyword, and discard data in any band that exceeds a given ppErrBits value.

This may be... slow. And awkward to implement. Let's, however, try. We may need to insert a `nan` step - i.e., scan each row, and if (e.g.) JPPERRBITS is above a max_flag value, set JAPERMAG3 (and JAPERMAG3ERR, JMHPNT, JMHPNTERR, any other columns depending on JAPERMAG3) to np.nan, before proceeding.

Oof.